## Grocery Store Competition
Brick-and-mortar grocery stores are always in a delicate dance with purchasing and sales forecasting. Predict a little over, and grocers are stuck with overstocked, perishable goods. Guess a little under, and popular items quickly sell out, leaving money on the table and customers fuming.

The problem becomes more complex as retailers add new locations with unique needs, new products, ever transitioning seasonal tastes, and unpredictable product marketing. Corporación Favorita, a large Ecuadorian-based grocery retailer, knows this all too well. They operate hundreds of supermarkets, with over 200,000 different products on their shelves.

Corporación Favorita has challenged the Kaggle community to build a model that more accurately forecasts product sales. They currently rely on subjective forecasting methods with very little data to back them up and very little automation to execute plans. They’re excited to see how machine learning could better ensure they please customers by having just enough of the right products at the right time.

[Corporación Favorita Grocery Sales Forecasting](https://www.kaggle.com/c/favorita-grocery-sales-forecasting)

* this is current, so we won't work on it as a group
* predict items on shelf based on
* oil prices, stores, locations
* ability to explain the problem is very important
* key: what are independent variables- how many units of each kind of product are sold on each store on each day during a 2 week period?
* dependent, info we have to predict it: how many units of each product of each store on each day was sold;
* what metadata is there (oil price) --> our relational dataset
* Stars schema https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data
* Snowflake schema: might have more info on the items
* Jeremy's notebook: tmp-grocery.ipynb

## 1. Imports

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [7]:
PATH = "data/grocery-sales/"
!ls {PATH}

holidays_events.csv  oil.csv		    stores.csv	train.csv
items.csv	     sample_submission.csv  test.csv	transactions.csv


## 2. Read data
* read in data
* limit memory = False --> use as much memory to figure out what kinds of data are here; you'll run out of memory here
* to limit the amount of space, create a dictionary for each column names

In [8]:
types = {'id': 'int64',
'item_nbr': 'int32',
'store_nbr': 'int8',
'unit_sales': 'float32',
'onpromotion': 'object'}

this dataset has 125 million rows

use head function to look at small amount of data; determine the data types and set it up in a dictionary called "types" (see above)

or read in small dataset and let pandas figure it out for you

can now read in data in less than 2 minutes

In [9]:
%%time
df_all = pd.read_csv(f'{PATH}train.csv', parse_dates=['date'], dtype = types,
                    infer_datetime_format = True)

CPU times: user 2min, sys: 3.37 s, total: 2min 4s
Wall time: 2min 4s


In [16]:
df_all.onpromotion.fillna(False, inplace=True)
df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

In [14]:
%time df_all.to_feather('tmp/raw_groceries')

CPU times: user 2.24 s, sys: 1.94 s, total: 4.17 s
Wall time: 3.06 s


In [15]:
%time df_all.describe(include='all')

CPU times: user 36.8 s, sys: 584 ms, total: 37.4 s
Wall time: 37.4 s


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.254970e+08,125497040,1.254970e+08,1.254970e+08,1.254970e+08,125497040
unique,NaN,1684,NaN,NaN,NaN,2
top,NaN,2017-07-01 00:00:00,NaN,NaN,NaN,False
freq,NaN,118194,NaN,NaN,NaN,96028767
first,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,6.274852e+07,NaN,2.746458e+01,9.727692e+05,8.554856e+00,NaN
std,3.622788e+07,NaN,1.633051e+01,5.205336e+05,2.360515e+01,NaN
min,0.000000e+00,NaN,1.000000e+00,9.699500e+04,-1.537200e+04,NaN
25%,3.137426e+07,NaN,1.200000e+01,5.223830e+05,2.000000e+00,NaN


可以看出Trainset的日期是从：

2013-01-01 00:00:00 - 2017-08-15 00:00:00

Testset from :

2017-08-16 00:00:00 - 2017-08-31 00:00:00 (one day later)


In [17]:
df_test = pd.read_csv(f'{PATH}test.csv', parse_dates=['date'], dtype = types,
                    infer_datetime_format = True)
df_test.onpromotion.fillna(False, inplace=True)
df_test.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_test.onpromotion = df_all.onpromotion.astype(bool)
df_test.describe(include='all')

,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,3370464
unique,NaN,16,NaN,NaN,1
top,NaN,2017-08-27 00:00:00,NaN,NaN,True
freq,NaN,210654,NaN,NaN,3370464
first,NaN,2017-08-16 00:00:00,NaN,NaN,NaN
last,NaN,2017-08-31 00:00:00,NaN,NaN,NaN
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN


In [18]:
df_all.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
125497035,125497035,2017-08-15,54,2089339,4.0,True
125497036,125497036,2017-08-15,54,2106464,1.0,True
125497037,125497037,2017-08-15,54,2110456,192.0,True
125497038,125497038,2017-08-15,54,2113914,198.0,True
125497039,125497039,2017-08-15,54,2116416,2.0,True


In [ ]:
df_all = pd.read_feather('tmp/raw_groceries')

$$NWRMSLE = \sqrt{ \frac{\sum_{i=1}^n w_i \left( \ln(\hat{y}_i + 1) - \ln(y_i +1)  \right)^2  }{\sum_{i=1}^n w_i}}$$
* in this competition，the **root mean squared log error** is the evaluation
* there are some **negative sales** that we should consider them to be 0


所以我们做如下变换：

In [19]:
df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [20]:
add_datepart(df_all, 'date')

#### Split validation set

In [21]:
def split_vals(a, n):
    return a[:n].copy(), a[n:].copy()

In [22]:
n_valid = len(df_test)
n_trn = len(df_all) - n_valid
train, valid = split_vals(df_all, n_trn)
train.shape, valid.shape

((122126576, 18), (3370464, 18))

In this case, I didn't need to run `train_cats()`, because all of my data types are already numerical.

If they weren't I would need to call `train_cast` and then I would need to call the `apply_cats` to apply the same categorical codes from the training set to the validation set：



In [ ]:
# train_cats(raw_train)
# apply_cats(raw_valid, raw_train)

`proc_df`: make a copy of the dataframe, grab the Y value, drop the dependent variable, from the original and then it's going to fix missing. so how do we fix missing? 
>if it's numeric then we fix it by basically saying let's first of all check that it does have some missing right? so if it does have some missing values then we're going to create a new column called `xxx_na`, and it's going to be a boolean column with a `1` anytime that was missing, and a `0` anytime it wasn't. Then replace the NAs(the missing) with the median. 

In [27]:
trn, y, nas_trn = proc_df(train, 'unit_sales')
val, y_val, nas_val = proc_df(valid, 'unit_sales')

## 3. Model


先sample 1000000的小数据集来试试rf：

In [31]:
set_rf_samples(1_000_000)

In [28]:
x = np.array(trn, dtype=np.float32)

In [30]:
x.shape

(122126576, 17)

In [29]:
def rmse(x, y): 
    return math.sqrt(((x - y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x), y), rmse(m.predict(val), y_val), m.score(x, y), m.score(val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

there's no relationship between the size of the dataset(x) and how long it takes to build the random forests(time). their relationship is between the number of estimators(20) multiplied by the sample size(1_000_000).


* there are 120 million records; we probably don't want to create a tree; would take a long time
* can start with 10K or 100K; Jeremy found 1 million is good size, runs in < 1 minute
* there is no relationship between how large a dataset is and how long it takes to build a random forest
* relationship is between number of estimators times sample size
* n_jobs=8 number of cores it will use; Jeremy ran it on computer that had 60 cores, so make it smaller
* n_jobs=-1 means use every single core
* Jeremy converted dataframe into array of float32; internally inside random forest code, they do that anyway
* by doing it once, saves time in the background to convert it to float

In [32]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=100, n_jobs=8)
%time m.fit(x, y)

CPU times: user 6min 19s, sys: 12.5 s, total: 6min 32s
Wall time: 1min


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=100, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=8,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

* `prun` runs profiler, tells you which line of code behind the scenes took the most time to run
* *profiler* is a software engineering tool
* cannot use oob_score when using set_rf_samples, because it will use 125 million - 1 million = 124 million too calculate oob_score, which will take forever
>因为oob_score是out of bag score，也就是说用trainset中所有没有被sample的data来test得到score
* wants validation set, which is the most recent date samples


In [34]:
%prun m.fit(x, y)

In [33]:
print_score(m)

[0.7650825024257072, 0.7656551271739643, 0.2473550581382178, 0.21977441813084242]


`[training RMSE , validation RMSE, training R^2, validation R^2, OOB R^2]`

In [35]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=10, n_jobs=8)
%time m.fit(x, y)
print_score(m)

CPU times: user 7min 16s, sys: 12.4 s, total: 7min 29s
Wall time: 1min 7s
[0.6855446149249788, 0.7021835899695322, 0.39571048986244917, 0.3437714104431948]


In [36]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, n_jobs=8)
%time m.fit(x, y)
print_score(m)

CPU times: user 7min 43s, sys: 12.8 s, total: 7min 56s
Wall time: 1min 12s
[0.6611483982829707, 0.6934118503161113, 0.4379544570093905, 0.360064335905226]


很明显上面的rf model都不是很work: `rmsle≈0.7`。

再看一下trainset，提供的信息太少了，我们还需要一些metadata（weather, store_location...）.

one way of tackling this kind of problem is to create lots and lots of new columns containing things like: average number of sales on holidays, average percent change in sale, or between January and February and so on and so forth ...



In [37]:
df_all.tail()

,id,store_nbr,item_nbr,unit_sales,onpromotion,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
125497035,125497035,54,2089339,1.609438,True,2017,8,33,15,1,227,False,False,False,False,False,False,1502755200
125497036,125497036,54,2106464,0.693147,True,2017,8,33,15,1,227,False,False,False,False,False,False,1502755200
125497037,125497037,54,2110456,5.262690,True,2017,8,33,15,1,227,False,False,False,False,False,False,1502755200
125497038,125497038,54,2113914,5.293305,True,2017,8,33,15,1,227,False,False,False,False,False,False,1502755200
125497039,125497039,54,2116416,1.098612,True,2017,8,33,15,1,227,False,False,False,False,False,False,1502755200


Trick：there's a kaggle kernel that points out that what you could do is just take the last two weeks and take the average sales the by `onpromotion` by `store_nbr` by `item_nbr`, and just submit that, and you come about 30th. 

**How to start from this model, and make it a tiny bit better?**

【Trick】there's one thing I'm going to let you use the *testset* for an addition and that is to calibrate your *validation set* so what Terrance did here was, he built 4 different models, some which he thought would be better than others and he submitted each of the four models to Kaggle to find out its score, and so the x-axis is the score the Kaggle produce on the leaderboard, and then on the y-axis he plotted the score on a particular *validation set* .

he was trying out to see whether this validation said look like it was going to be any good, so if your validation set is good then the relationship between the leaderboards for by the testing score and your validation set score should lie in a straight line, ideally, it'll actually lie on the $y = x$ line, but honestly that doesn't matter too much as long as relatively speaking it tells you which models are better than which other models, then you know which model is the best right? and you know how it's going to perform on the testset, because you know the linear relationship between two things okay?
 


<img src="images/validation_4_scores.png" width="70%">
